In [1]:
from tl_tools import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12288)]
        )
        print("GPU memory limit set to 12GB")
    except RuntimeError as e:
        print(f"Error setting GPU memory limit: {e}")

GPU memory limit set to 12GB


In [2]:
setup_mixed_precision()
train_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/train'

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True ,vertical_flip=True)
train =train_datagen.flow_from_directory(train_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))


X_train, y_train, X_val, y_val = train_val_split(train, val_split= 0.3)

Mixed precision policy set to: mixed_float16

Found 4980 images belonging to 6 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model


def basic_residual_block_v2(x, filters, stride=1):
    """ Basic residual block for ResNetV2 with pre-activation (BatchNorm -> ReLU -> Conv) """
    shortcut = x
    
    #first bacthcnorm -> relu -> conv layer
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 3, strides=stride, padding='same')(x)
    
    #second layer
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    
    #change in shortcut if necessary
    if stride!= 1:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
    
    #skip connection
    x = layers.Add()([x, shortcut])
    return x





def custom_resnet34v2(input_shape=(256, 256, 3), num_classes=6):
    """
    Creates a ResNet34 v2 like model.
    
    Args:
    - input_shape: Expected input shape (height, width, channels). Default: (256, 256, 3)
    - num_classes: Number of classes for the classification task. Default: 6
    """
   
    inputs = layers.Input(shape=input_shape)
    
    #initial convlayer and maxpool
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    #1: 3 residual blocks with 64 filters
    for _ in range(3):
        x = basic_residual_block_v2(x, 64)
    
    #2: 4 residual blocks with 128 filters
    for i in range(4):
        stride = 2 if i == 0 else 1  # Downsample at the start of the stage
        x = basic_residual_block_v2(x, 128, stride=stride)
    
    #3: 6 residual blocks with 256 filters
    for i in range(6):
        stride = 2 if i == 0 else 1
        x = basic_residual_block_v2(x, 256, stride=stride)
    
    #4: 3 residual blocks with 512 filters
    for i in range(3):
        stride = 2 if i == 0 else 1
        x = basic_residual_block_v2(x, 512, stride=stride)
    
    #global average pooling (preparing for FC)
    x = layers.GlobalAveragePooling2D()(x)
    
    #FC1
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    #FC2
    x = layers.Dense(32, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    #output layer
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='ResNet34V2_like_with_custom_output')
    
    return model

# Create the model
resnet_34v2_custom = custom_resnet34v2()

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
resnet_34v2_custom.compile(optimizer=Adam(learning_rate = 0.001, clipnorm= 1.0),
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=True, start_from_epoch=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

In [6]:
save_freq = (10 * 110) 

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:03d}.keras'),
    save_weights_only=False, 
    save_best_only=False,     
    save_freq= save_freq, #saves after every 5 * 110 batches per epoch (5 epochs), experienced memory issues
    verbose=1
)


#history =  resnet_34v2_custom.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1, 
#                            callbacks= [early_stopping, reduce_lr, checkpoint_callback])

In [7]:
latest_checkpoint = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/checkpoints/model_epoch_030.keras'
resnet_34v2_custom = tf.keras.models.load_model(latest_checkpoint)
lr = float(tf.keras.backend.get_value(resnet_34v2_custom.optimizer.learning_rate))
print(f'Current learning rate: {lr}')

resnet_34v2_custom.compile(
    optimizer = Adam(learning_rate= lr),
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy','F1Score']
)

Current learning rate: 0.0005000000237487257


In [9]:
history = resnet_34v2_custom.fit(X_train, y_train, validation_data= (X_val, y_val), epochs = 100, initial_epoch= 30, 
                               callbacks = [checkpoint_callback, reduce_lr, early_stopping])

Epoch 31/100


2024-10-23 16:00:14.744448: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


110/110 ━━━━━━━━━━━━━━━━━━━━ 93s 706ms/step - F1Score: 0.7768 - accuracy: 0.7813 - loss: 0.6347 - val_F1Score: 0.7282 - val_accuracy: 0.7276 - val_loss: 0.7250 - learning_rate: 5.0000e-04
Epoch 32/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 72s 656ms/step - F1Score: 0.8134 - accuracy: 0.8181 - loss: 0.5360 - val_F1Score: 0.6429 - val_accuracy: 0.6579 - val_loss: 1.1806 - learning_rate: 5.0000e-04
Epoch 33/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 72s 653ms/step - F1Score: 0.7820 - accuracy: 0.7856 - loss: 0.5956 - val_F1Score: 0.7948 - val_accuracy: 0.7947 - val_loss: 0.5555 - learning_rate: 5.0000e-04
Epoch 34/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 72s 651ms/step - F1Score: 0.7940 - accuracy: 0.7957 - loss: 0.5964 - val_F1Score: 0.6526 - val_accuracy: 0.6667 - val_loss: 1.0036 - learning_rate: 5.0000e-04
Epoch 35/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 72s 651ms/step - F1Score: 0.8068 - accuracy: 0.8111 - loss: 0.5419 - val_F1Score: 0.6631 - val_accuracy: 0.6789 - val_loss: 0.9037 - learning_rate: 5.0000e-04
Epoch 36

In [11]:
resnet_34v2_custom.save('resnet34v2.keras') 